# Analysis of the characterstics of wireless channel for Rural Macro (RMa) terrain

* The `Rural Macro` or `"RMa"` deployment `scenario/terrain` focuses on larger and continuous coverage.
* The key characterstics of this scenario are continuous wide area coverage supporting high speed vehicles.

* In this tutorial, we will analyze the performance of channel Model under the propagation scenario Rural Macro or "RMa" for a Hexagonal Base Station (BS) Layout.
* For a given number of BSs and UEs we generate cluster level channel coefficients corresponding to every link being simulated. 

* We first import the necessary libraries then followed by creating objects of classes `AntennaArrays`, `NodeMobility`, 
  and `SimulationLayout` respectively.

The content of the tutorial is as follows:

## Table of Contents

* [Python Libraries](#import-python-libraries)
* [5G Toolkit Libraries](#import-5G-toolkit-libraries)
* [Simulation Parameters](#simulation-parameters)
* [Antenna Array Objects](#antenna-array-obj)
* [Node Mobility Objects](#node-mobility-obj)
* [Simulation Layout Object](#simulation-layout-obj)
* [Channel Parameters](#lsp-ssp)
* [Channel Coefficients](#clusterlevel-channelcoeff)








## Python Libraries

In [1]:
# importing necessary python libraries
%matplotlib widget

import numpy              as np
import matplotlib.pyplot  as plt
import matplotlib         as mpl
import matplotlib.patches as patches

## 5G Toolkit Libraries

In [2]:
# importing necessary modules for simulating channel model
import sys
sys.path.append("../../")
from toolkit5G.channelModels.nodeMobility.nodeMobility import NodeMobility
from toolkit5G.channelModels.antennaArrays.antennaArray import AntennaArrays
from toolkit5G.channelModels.simulationLayout.simulationLayout import SimulationLayout
from toolkit5G.channelModels.channelGenerator.coefficientGenerator import CoefficientGenerator

ModuleNotFoundError: No module named 'ChannelModels'

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

## Simulation Parameters

The simulation parameters are defined as follows
  * ``propTerrain`` defines propagation scenario or terrain for BS-UE, UE-UE, BS-BS links
  * ``carrierFrequency`` defines  array of carrier frequencies in GHz
  * ``nBSs`` defines number of Base Stations (BSs)
  * ``nUEs`` defines number of User Equipments (UEs) 
  * ``nSnapShots`` defines number of SnapShots

In [ ]:
# Simulation Parameters
propTerrain      = np.array(["RMa"])# Propagation Scenario or Terrain for BS-UE links
carrierFrequency = np.array([3,5])  # Array of two carrier frequencies in GHz
nBSs             = 19               # number of BSs 
nUEs             = 50              # number of UEs 
nSnapShots       = 2                # number of SnapShots

## Antenna Array Objects

The following steps describe the procedure to simulate a vector or a numpy array of AntennaArrays Objects at each carrier frequency
both at Tx and Rx side:


In [ ]:
# Antenna Array at UE side
# assuming antenna element type to be "OMNI"
# with 2 panel and 2 single polarized antenna element per panel.
numCarriers = carrierFrequency.shape[0]
ueAntArray  = np.empty(numCarriers, dtype=object)
for i in range(carrierFrequency.size):
    ueAntArray[i] = AntennaArrays(antennaType     = "OMNI", 
                                  centerFrequency = carrierFrequency[i],
                                  arrayStructure  = np.array([1,1,2,2,1]))
    ueAntArray[i]() 

The user can display the Rx antenna radiation pattern in 3D as follows:
For instance at 3 GHz carrier frequency, the radiation pattern can be plotted with a call to a method `displayAntennaRadiationPattern()` of the class `AntennaArrays`

In [ ]:
# Radiation Pattern of Rx antenna element 
ueAntArray[0].displayAntennaRadiationPattern()

In [ ]:
# Antenna Array at BS side
# assuming antenna element type to be "3GPP_38.901", a parabolic antenna 
# with 4 panel and 4 single polarized antenna element per panel.
numCarriers    = carrierFrequency.shape[0]
bsAntArray     = np.empty(numCarriers, dtype=object)
for i in range(carrierFrequency.size):
    bsAntArray[i] = AntennaArrays(antennaType     = "3GPP_38.901", 
                                  centerFrequency = carrierFrequency[i],
                                  arrayStructure  = np.array([1,1,4,4,1]))
    bsAntArray[i]()

In [ ]:
# Radiation Pattern of Tx antenna element 
bsAntArray[0].displayAntennaRadiationPattern()

## Node Mobility Objects

This subsection provides the following steps to simulate the mobility of each node

In [ ]:
# NodeMobility parameters
# assuming that all the BSs are static and all the UEs are mobile.
timeInst   = np.linspace(start = 0, stop = 10, num=nSnapShots, dtype=np.float32) # time values at each snapshot.
UEroute    = NodeMobility("randomWalk", nUEs, timeInst)

## Simulation Layout Object

The following code snippet provides the parameters to instantiate the object of the class SimulationLayout.

In [ ]:
# Layout Parameters
isd                  = 1732        # inter site distance
minDist              = 35          # min distance between each UE and BS 
ueHt                 = 1.5         # UE height
bsHt                 = 35          # BS height
bslayoutType         = "Hexagonal" # BS layout type
ueDropType           = "Hexagonal" # UE drop type
htDist               = "equal"     # UE height distribution
ueDist               = "equal"     # UE Distribution per site
nSectorsPerSite      = 3           # number of sectors per site
maxNumFloors         = 1           # Max number of floors in an indoor object
minNumFloors         = 1           # Min number of floors in an indoor object
heightOfRoom         = 3           # height of room or ceiling in meters
indoorUEfract        = 0.5         # Fraction of UEs located indoor
lengthOfIndoorObject = 3           # length of indoor object typically having rectangular geometry 
widthOfIndoorObject  = 3           # width of indoor object
forceLOS             = False       # boolen flag if true forces every link to be in LOS state 

In [ ]:
# simulation layout object                                 
>>> simLayoutObj = SimulationLayout(numOfBS = nBSs,
...                                 numOfUE = nUEs,
...                                 heightOfBS = bsHt,
...                                 heightOfUE = ueHt, 
...                                 ISD = isd,
...                                 layoutType = bslayoutType,
...                                 ueDropMethod = ueDropType, 
...                                 UEdistibution = ueDist,
...                                 UEheightDistribution = htDist,
...                                 numOfSectorsPerSite = nSectorsPerSite,
...                                 minUEBSDistance = minDist,
                                    ueRoute = UEroute) 
>>> simLayoutObj(terrain = propTerrain, 
...              carrierFreq = carrierFrequency, 
...              ueAntennaArray = ueAntArray,
...              bsAntennaArray = bsAntArray,
...              indoorUEfraction = indoorUEfract,
...              lengthOfIndoorObject = lengthOfIndoorObject,
...              widthOfIndoorObject = widthOfIndoorObject,
...              forceLOS = forceLOS)
# displaying the topology of simulation layout
>>> simLayoutObj.display2DTopology()

## Channel Parameters

* This subsection provides the steps to obtain all the cluster level channel parameters, which includes both `Large Scale Parameters (LSPs)`
  and `Small Scale Parameters (SSPs)`. 
* We plot the distribution of LSPs such as `Path Loss (PL)`, `Delay Spread (DS)` and `Angular Spreads` both in Azimuth and Zenith directions,
  and from SSPs we plot the distribution of `cluster powers (Pn)`.
  
Cluster level channel parameters are obtained as follows:

In [ ]:
# channel parameters
paramGenObj = simLayoutObj.getParameterGenerator()

### PathLoss
The following code snippet plots the `cdf/pdf of PathLoss (PL) in dB scale` and `PathLoss (PL) vs 3D distances (d3D)`.

In [ ]:

n_bins = 50
# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.pathLoss, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
  
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

fig, ax = plt.subplots(1, 2)
# set the spacing between subplots
fig.tight_layout()
# plotting PDF and CDF
ax[0].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0].plot(bins_count[1:], cdf, label="CDF")
ax[0].set_title("CDF|PDF of PathLoss in dB")
ax[0].set_ylabel("$F(PL)|f(PL)$")
ax[0].set_xlabel("Bin Count")
ax[0].legend()

# numAverages = 100
numAverages = int(simLayoutObj.d3D[:,0:simLayoutObj.numOfBS,simLayoutObj.numOfBS:simLayoutObj.numOfBS+simLayoutObj.numOfUE].flatten().shape[0]/10)
sortInd  = np.argsort(simLayoutObj.d3D[:,0:simLayoutObj.numOfBS,simLayoutObj.numOfBS:simLayoutObj.numOfBS+simLayoutObj.numOfUE].flatten().flatten())
distance = np.mean(simLayoutObj.d3D[:,0:simLayoutObj.numOfBS,simLayoutObj.numOfBS:simLayoutObj.numOfBS+simLayoutObj.numOfUE].flatten()[sortInd].reshape(-1, numAverages), axis = 1)
pathLoss = np.mean(paramGenObj.pathLoss[0].flatten()[sortInd].reshape(-1, numAverages), axis = 1)
ax[1].plot(distance, pathLoss, color="green", label="PL in dB")
ax[1].set_title("PathLoss in dB")
ax[1].set_ylabel("PL")
ax[1].set_xlabel("d3D")
ax[1].legend()
ax[1].set_ylim([paramGenObj.pathLoss.min(), paramGenObj.pathLoss.max()])

# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
# Show plot
plt.show()

### DelaySpread

The following code snippet generate Delay Spread

In [ ]:
paramGenObj.DS.min(), paramGenObj.DS.max(), paramGenObj.DS.mean()

In [ ]:
n_bins = 50

# plotting PDF and CDF
fig, ax = plt.subplots(1, 2)

# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.DS, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.DS.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[0].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0].plot(bins_count[1:], cdf, label="CDF")
ax[0].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value")
ax[0].set_title("CDF|PDF of $\Delta \\tau$")
ax[0].set_ylabel("$f(\Delta \\tau)$")
ax[0].set_xlabel("$\Delta \\tau$")
ax[0].legend(loc=1)

#********************************************************************

# getting data of the histogram
count, bins_count = np.histogram(np.log10(paramGenObj.DS), bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu  = np.log10(paramGenObj.DS).mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[1].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[1].plot(bins_count[1:], cdf, label="CDF")
ax[1].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax[1].legend(loc=1)
ax[1].set_title("CDF|PDF of $log_{10}(\Delta \\tau)$")
ax[1].set_ylabel("$f(log_{10}(\Delta \\tau))$")
ax[1].set_xlabel("$log_{10}(\Delta \\tau)$")
# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
# Show plot
plt.show()

### Angular Spreads

One can plot the Angular Spreads using the following script:

In [ ]:
n_bins = 50

# plotting PDF and CDF
fig, ax = plt.subplots(2, 4)

#*************************  ASA  *******************************
# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.ASA, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.ASA.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[0,0].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0,0].plot(bins_count[1:], cdf, label="CDF")
ax[0,0].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value")
ax[0,0].set_title("CDF|PDF of Arrival $\Delta \\phi$", fontsize = 12)
ax[0,0].set_ylabel("$f_{A}(\Delta \\phi)$", fontsize = 12)
ax[0,0].set_xlabel("$\Delta \\phi$", fontsize = 12)
ax[0,0].legend(loc=1)

#********************************************************************

# getting data of the histogram
count, bins_count = np.histogram(np.log10(paramGenObj.ASA), bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu  = np.log10(paramGenObj.ASA).mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[0,1].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0,1].plot(bins_count[1:], cdf, label="CDF")
ax[0,1].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax[0,1].legend(loc=1)
ax[0,1].set_title("CDF|PDF of Arrival $log_{10}(\Delta \\phi)$", fontsize = 12)
ax[0,1].set_ylabel("$f_{A}(log_{10}(\Delta \\phi))$", fontsize = 12)
ax[0,1].set_xlabel("$log_{10}(\Delta \\phi)$", fontsize = 12)

#*************************  ASD  *******************************

# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.ASD, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.ASD.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[0,2].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0,2].plot(bins_count[1:], cdf, label="CDF")
ax[0,2].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value")
ax[0,2].set_title("CDF|PDF of Departure $\Delta \\phi$")
ax[0,2].set_ylabel("$f_{D}(\Delta \\phi)$", fontsize = 12)
ax[0,2].set_xlabel("$\Delta \\phi$", fontsize = 12)
ax[0,2].legend(loc=1)

#********************************************************************

# getting data of the histogram
count, bins_count = np.histogram(np.log10(paramGenObj.ASD), bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu  = np.log10(paramGenObj.ASD).mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[0,3].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[0,3].plot(bins_count[1:], cdf, label="CDF")
ax[0,3].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax[0,3].legend(loc=1)
ax[0,3].set_title("CDF|PDF of Departure $log_{10}(\Delta \\phi)$")
ax[0,3].set_ylabel("$f_{D}(log_{10}(\Delta \\phi))$", fontsize = 12)
ax[0,3].set_xlabel("$log_{10}(\Delta \\phi)$", fontsize = 12)

#*************************  ZSA  *******************************
# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.ZSA, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.ZSA.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[1,0].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[1,0].plot(bins_count[1:], cdf, label="CDF")
ax[1,0].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value")
ax[1,0].set_title("CDF|PDF of Arrival $\Delta \\theta$")
ax[1,0].set_ylabel("$f_{A}(\Delta \\theta)$", fontsize = 12)
ax[1,0].set_xlabel("$\Delta \\theta$", fontsize = 12)
ax[1,0].legend(loc=1)

#********************************************************************

# getting data of the histogram
count, bins_count = np.histogram(np.log10(paramGenObj.ZSA), bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu  = np.log10(paramGenObj.ZSA).mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[1,1].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[1,1].plot(bins_count[1:], cdf, label="CDF")
ax[1,1].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax[1,1].legend(loc=1)
ax[1,1].set_title("CDF|PDF of Arrival $log_{10}(\Delta \\theta)$")
ax[1,1].set_ylabel("$f_{A}(log_{10}(\Delta \\theta))$", fontsize = 12)
ax[1,1].set_xlabel("$log_{10}(\Delta \\theta)$", fontsize = 12)

#*************************  ZSD  *******************************

# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.ZSD, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.ZSD.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[1,2].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[1,2].plot(bins_count[1:], cdf, label="CDF")
ax[1,2].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value")
ax[1,2].set_title("CDF|PDF of Departure $\Delta \\theta$", fontsize = 12)
ax[1,2].set_ylabel("$f_{D}(\Delta \\theta)$", fontsize = 12)
ax[1,2].set_xlabel("$\Delta \\theta$", fontsize = 12)
ax[1,2].legend(loc=1)

#********************************************************************

# getting data of the histogram
count, bins_count = np.histogram(np.log10(paramGenObj.ZSD), bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu  = np.log10(paramGenObj.ZSD).mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax[1,3].plot(bins_count[1:], pdf, color="red", label="PDF")
ax[1,3].plot(bins_count[1:], cdf, label="CDF")
ax[1,3].axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax[1,3].legend(loc=1)
ax[1,3].set_title("CDF|PDF of Departure $log_{10}(\Delta \\theta)$", fontsize = 12)
ax[1,3].set_ylabel("$f_{D}(log_{10}(\Delta \\theta))$", fontsize = 12)
ax[1,3].set_xlabel("$log_{10}(\Delta \\theta)$", fontsize = 12)


# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

# Show plot
plt.show()

### Cluster Powers
The following script plot the statistical distribution of cluster power Pn

In [ ]:
n_bins = 50

# plotting PDF and CDF
fig, ax = plt.subplots(1, 1)

# getting data of the histogram
count, bins_count = np.histogram(paramGenObj.Pn, bins=n_bins)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
mu = paramGenObj.Pn.mean()
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)

ax.plot(bins_count[1:], pdf, color="red", label="PDF")
ax.plot(bins_count[1:], cdf, label="CDF")
ax.axvline(x = mu, ymin = 0, ymax = 1, color ='green', label="mean value: "+str(mu))
ax.set_title("CDF|PDF of $P_{n}$")
ax.set_ylabel("$f(P_{n})$")
ax.set_xlabel("$P_{n}$")
ax.legend()


plt.show()

## Channel Coefficients

Cluster level channel coefficients can be simulated using the following code snippet

In [ ]:
# Instantiating CoefficientGenerator class with the following parameters
coeffGenObj = CoefficientGenerator(fc = carrierFrequency,
                                   terrain = propTerrain,
                                   locBS = simLayoutObj.BSLocations + simLayoutObj.BStracks,
                                   locUE = simLayoutObj.UELocations + simLayoutObj.UEtracks,
                                   txAntArrayObjVec = bsAntArray,
                                   rxAntArrayObjVec = ueAntArray,
                                   velocityVector   = simLayoutObj.UEvelocityVector,
                                   timeInstances    = timeInst,      
                                   d3D   = simLayoutObj.d3D,
                                   d2D   = simLayoutObj.d2D)
# Passing link state vector as input                               
coeffGenObj(simLayoutObj.linkState) 

The following code shows how to access cluster level channel coefficients and delays.
* `channelCoeff` is a numpy array of shape (`Nf`,`Ns`,`Nbs`,`Nue`,`Np`,`Nrxant`,`Ntxant`) and
* `channelDelays` a numpy array of shape (`Nf`,`Ns`,`Nbs`,`Nue`,`Np`) respectively
Where 
* `Nf` correspond to number of carrier frequencies in simulation
* `Ns` correspond to number of Snap-Shots
* `Nbs` correspond to number of BSs
* `Nue` correspond to number of UEs
* `Np` correspond to number of paths or clusters
* `Nrxant` correspond to number of Rx Antenna elements
* `Ntxant` correspond to number of Tx Antenna elements

Since the antenna elements at both Tx and Rx experience same delay, we exclude these dimensions when implementing `channelDelays` corresponding to the class `CoefficientGenerator`.       
      
      

In [ ]:
# Printing channel coefficients
coeffGenObj.channelCoeff.shape 
#(Nf,Ns,Nbs,Nue,Np,Nrxant,Ntxant)

In [ ]:
# Printing channel delays
coeffGenObj.channelDelays.shape
#(Nf,Ns,Nbs,Nue,Np)